In [ ]:
import numpy as np 
from keras.datasets import mnist #Keras is used only to get MNIST data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_train = Y_train.reshape(60000, 1).T 
b = np.zeros((60000, 10))
b[np.arange(60000), Y_train] = 1
Y_train = b.T #To convert to one hot matrix
print(Y_train.shape)
X_train = X_train.reshape(60000, 28, 28, 1)/255 #Divided by 255 to normalize the input data
X_train = X_train.reshape(X_train.shape[0], -1).T #We use -1 because numpy will itself figure out what -1 is by looking at length of tha array and the remaining dimensions
print(X_train.shape)
Y_test = Y_test.reshape(10000, 1).T
b = np.zeros((10000, 10))
b[np.arange(10000), Y_test] = 1
Y_test = b.T
print(Y_test.shape)
X_test = X_test.reshape(10000, 28, 28, 1)/255
X_test = X_test.reshape(X_test.shape[0], -1).T
print(X_test.shape)
def relu(z):
    z = z * (z > 0)
    return z
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)
learning_rate = 1
m = 60000

In [ ]:
W1 = np.random.randn(64, 784) * 0.01 #0.01 is multiplied to make gradient descent faster
#W's are initialized randomly in order to break symmetry
b1 = np.zeros((64, 1))
W2 = np.random.randn(32, 64) * 0.01
b2 = np.zeros((32, 1))
W3 = np.random.randn(10, 32) * 0.01
b3 = np.zeros((10, 1))
for i in range(2000):
    #Forward propagation
    Z1 = np.matmul(W1, X_train) + b1
    A1 = relu(Z1)
    Z2 = np.matmul(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.matmul(W3, A2) + b3
    A3 = softmax(Z3)
    cost = np.sum(np.multiply(np.log(A3), Y_train))/ (-60000)
    print(cost)
    #Backward propagation
    dZ3 = A3 - Y_train
    dW3 = (1./60000) * np.matmul(dZ3, A2.T)
    db3 = (1./60000) * np.sum(dZ3, axis=1, keepdims=True)
    dA2 = np.matmul(W3.T, dZ3)
    dZ2 = dA2 * (1 * (Z2 > 0))
    dW2 = (1./60000) * np.matmul(dZ2, A1.T)
    db2 = (1./60000) * np.sum(dZ2, axis=1, keepdims=True)
    dA1 = np.matmul(W2.T, dZ2)
    dZ1 = dA1 * (1 * (Z1 > 0))
    dW1 = (1./m) * np.matmul(dZ1, X_train.T)
    db1 = (1./m) * np.sum(dZ1, axis=1, keepdims=True)
    W3 = W3 - learning_rate * dW3
    b3 = b3 - learning_rate * db3
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1

In [ ]:
#Finding out the accuracy
Z1 = np.matmul(W1, X_test) + b1
A1 = relu(Z1)
Z2 = np.matmul(W2, A1) + b2
A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)
predictions = np.argmax(A2, axis=0) #np.argmax axis = 0 return the values of the maximum indices along the columns
labels = np.argmax(Y_test, axis=0)
p = np.zeros((10000,))
for i in range(10000):
    if predictions[i] == labels[i]:
        p[i] = 1
    else:
        p[i] = 0
count = 0
for i in range(10000):
    if p[i] == 1:
        count = count + 1
accuracy = count / 100
print('The accuracy is ' + str(accuracy) + ' %.')